# Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 

**Основное**
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

### Решение

In [498]:
import pandas as pd
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

#### Загрузка данных и схема валидации

In [499]:
data_train = pd.read_csv('../raw_data/retail_train.csv')
item_features = pd.read_csv('../raw_data/product.csv')
user_features = pd.read_csv('../raw_data/hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data_train[data_train['week_no'] < data_train['week_no'].max() - 
                              (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_train_lvl_2 = data_train[(data_train['week_no'] >= data_train['week_no'].max() - 
                               (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data_train['week_no'] < data_train['week_no'].max() - (val_lvl_2_size_weeks))]
data_val_lvl_2 = data_train[data_train['week_no'] >= data_train['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


#### Первичная фильтрация данных

In [500]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


#### Обучение модели первого уровня

In [501]:
K = 5 # Число рекомендаций

In [502]:
recommender = MainRecommender(data_train_lvl_1)

#### Получаем список актуальных покупок и кандидатов для data_train_lvl_2

In [503]:
def get_actual(data):
    # Получение актуальных покупок
    result = data.groupby('user_id')['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    return result

In [504]:
def get_candidates(model, data):
    # Получение кандидатов на покупку
    data['candidates'] = data['user_id'].apply(lambda x: model.get_own_recommendations(x, N=50))
    return data

In [505]:
result_lvl_2 = get_actual(data_train_lvl_2)
result_lvl_2 = get_candidates(recommender, result_lvl_2)
result_lvl_2.head(2)

,user_id,actual,candidates
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 877391, 9655212, 88..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[911974, 1076580, 1103898, 5567582, 1056620, 9..."


#### Размечаем данные для второй модели

In [506]:
def get_X_data(data):
    # Формирование X данных
    s = data.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'item_id'
    return data.drop(['actual', 'candidates'], axis=1).join(s)

In [507]:
def mark_target(data, result):
    # Разметка данных
    users_lvl_2 = get_X_data(result)
    targets_lvl_2 = data[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1  # тут только покупки 
    targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
    targets_lvl_2['target'].fillna(0, inplace= True)
    
    return targets_lvl_2

In [508]:
X_y = mark_target(data_train_lvl_2, result_lvl_2)
X_y.head(2)

,user_id,item_id,target
0,1,856942,1.0
1,1,856942,1.0


#### Новые фичи

In [509]:
def featuring(data_train, data_target):
    # Добавляем внешние данные
    data_target = data_target.merge(item_features, on='item_id', how='left')
    data_target = data_target.merge(user_features, on='user_id', how='left')

    # Средний чек пользователя
    user_mean_price = pd.DataFrame()
    user_mean_price[['user_id', 'sum_sales']] = data_train.groupby(['user_id'])['sales_value'].sum().reset_index()\
    [['user_id', 'sales_value']]
    user_mean_price['sum_quantity'] = data_train.groupby(['user_id'])['quantity'].sum().reset_index()['quantity']
    user_mean_price['user_mean_price'] = user_mean_price['sum_sales'] / user_mean_price['sum_quantity']
    user_mean_price = user_mean_price.drop(['sum_sales', 'sum_quantity'], axis=1)

    # Средняя сумма покупок пользователей в разном возрасте
    user_age_sum_sales = pd.DataFrame()
    user_age_sum_sales[['age_desc', 'user_age_sum_sales']] = \
    data_train.merge(user_features, on='user_id', how='left').groupby(['age_desc'])\
    ['sales_value'].mean().reset_index()[['age_desc', 'sales_value']]
    
    # Средняя цена товара
    item_mean_price = pd.DataFrame()
    item_mean_price[['item_id', 'sum_sales']] = data_train.groupby(['item_id'])['sales_value'].sum().reset_index()\
    [['item_id', 'sales_value']]
    item_mean_price['sum_quantity'] = data_train.groupby(['item_id'])['quantity'].sum().reset_index()['quantity']
    item_mean_price['item_mean_price'] = item_mean_price['sum_sales'] / item_mean_price['sum_quantity']
    item_mean_price = item_mean_price.drop(['sum_sales', 'sum_quantity'], axis=1)
    
    # Среднее число продаж товара
    item_mean_quantity = pd.DataFrame()
    item_mean_quantity[['item_id', 'item_mean_quantity']] = data_train.groupby(['item_id'])\
    ['quantity'].mean().reset_index()[['item_id', 'quantity']]

    # Средняя сумма покупок пользователем в категории 
    user_department_mean_sales = pd.DataFrame()
    user_department_mean_sales[['user_id', 'department', 'user_department_mean_sales']] = \
    data_train.merge(item_features, on='item_id', how='left').groupby(['user_id', 'department'])\
    ['sales_value'].mean().reset_index()

    # Сумма покупок пользователем товара
    user_item_sum_sales = pd.DataFrame()
    user_item_sum_sales[['user_id', 'item_id', 'user_item_sum_sales']] = \
    data_train.groupby(['user_id', 'item_id'])['sales_value'].sum().reset_index()
    
    # Добавляем новые фичи в датасет
    data_target = data_target.merge(user_mean_price, on='user_id', how='left')
    data_target = data_target.merge(user_age_sum_sales, on='age_desc', how='left')
    data_target = data_target.merge(item_mean_price, on='item_id', how='left')
    data_target = data_target.merge(item_mean_quantity, on='item_id', how='left')
    data_target = data_target.merge(user_department_mean_sales, on=['user_id', 'department'], how='left')
    data_target = data_target.merge(user_item_sum_sales, on=['user_id', 'item_id'], how='left')
    
    return data_target

In [510]:
X_y = featuring(data_train_lvl_2, X_y)

#### Разбиваем на test и train и обучаем

In [511]:
cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product', 
             'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc', 'hh_comp_desc', 
             'household_size_desc', 'kid_category_desc']

In [512]:
def prepare_X(X):
    # Подготовка X данных перед обучением/предсказанием
    X[cat_feats] = X[cat_feats].astype('category')
    return X

In [513]:
def split_train_test(data):
    # Разделение на X и y
    X_train = data.drop('target', axis=1)
    X_train = prepare_X(X_train)
    y_train = data['target']    
    return X_train, y_train

In [514]:
X_train, y_train = split_train_test(X_y)

In [515]:
lgb = LGBMClassifier(objective='binary', max_depth=7, num_leaves=128)
lgb.fit(X_train, y_train)

LGBMClassifier(max_depth=7, num_leaves=128, objective='binary')

#### Берем топ-k предсказаний для data_train_lvl_2, ранжированных по вероятности, для каждого юзера

In [516]:
def get_model2(model, X, k):
    # Получение данных модели второго уровня
    X['preds'] = model.predict_proba(X)[:, 1]
    result = X.sort_values(['user_id', 'preds'], ascending=False).groupby('user_id').head(k).\
        groupby('user_id')['item_id'].unique().reset_index()
    result.rename(columns={'item_id': 'model2'}, inplace=True)
    return result

In [517]:
model2 = get_model2(lgb, X_train, K)
model2.head(2)

,user_id,model2
0,1,"[856942, 9297615, 5577022]"
1,2,"[1103898, 911974, 1076580, 5567582, 1056620]"


#### Находим рекомендации по первой модели

In [518]:
def get_model1(model, data, k):
    # Получение данных модели первого уровня
    model1 = data.copy()
    model1['model1'] = model1['user_id'].apply(lambda x: model.get_own_recommendations(x, N=k))
    return model1[['user_id', 'model1']]

In [519]:
model1 = get_model1(recommender, result_lvl_2, K)
model1.head(2)

,user_id,model1
0,1,"[856942, 9297615, 5577022, 877391, 9655212]"
1,2,"[911974, 1076580, 1103898, 5567582, 1056620]"


#### Сливаем всё в одну таблицу

In [520]:
result_lvl_2 = result_lvl_2.merge(model1, on='user_id', how='left')
result_lvl_2 = result_lvl_2.merge(model2, on='user_id', how='left')
result_lvl_2.head(2)

,user_id,actual,candidates,model1,model2
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[856942, 9297615, 5577022, 877391, 9655212]","[856942, 9297615, 5577022]"
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[911974, 1076580, 1103898, 5567582, 1056620]","[1103898, 911974, 1076580, 5567582, 1056620]"


#### Вычисляем метрики на data_train_lvl_2

In [521]:
def get_precision_at_k(data, field, k):
    # Вычисление усреднённого значения precision@k
    return data.apply(lambda x: precision_at_k(x[field], x['actual'], k), axis=1).mean()

In [522]:
print(' precision@k для модели первого уровня (data_train_lvl_2):', get_precision_at_k(result_lvl_2, 'model1', K), '\n',
      'precision@k для модели второго уровня (data_train_lvl_2):', get_precision_at_k(result_lvl_2, 'model2', K))

 precision@k для модели первого уровня (data_train_lvl_2): 0.17697307335190343 
 precision@k для модели второго уровня (data_train_lvl_2): 0.5507737542556483


### Получим расчет для data_val_lvl_2

In [523]:
# Первичная фильтрация данных
data_train_1_2 = pd.concat([data_train_lvl_1, data_train_lvl_2])
data_train_1_2 = prefilter_items(data_train_1_2, item_features=item_features, take_n_popular=5000)
# Обучение модели первого уровня
recommender = MainRecommender(data_train_1_2)
# Получаем список актуальных покупок и кандидатов для data_val_lvl_2
result_lvl_2 = get_actual(data_val_lvl_2)
result_lvl_2 = get_candidates(recommender, result_lvl_2)
# Формируем данные для второй модели
X = get_X_data(result_lvl_2)
# Фичи
X = featuring(data_val_lvl_2, X)
# Предобработка данных перед предсказанием модели 2
X = prepare_X(X)
# Берем топ-k предсказаний второй модели для data_val_lvl_2, ранжированных по вероятности, для каждого юзера
model2 = get_model2(lgb, X, K)
# Находим рекомендации по первой модели
model1 = get_model1(recommender, result_lvl_2, K)
# Сливаем всё в одну таблицу
result_lvl_2 = result_lvl_2.merge(model1, on='user_id', how='left')
result_lvl_2 = result_lvl_2.merge(model2, on='user_id', how='left')
result_lvl_2.head(2)

,user_id,actual,candidates,model1,model2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 9297615, 5577022, 877391, 8293439, 96...","[856942, 9297615, 5577022, 877391, 8293439]","[856942, 9297615, 5577022, 8293439, 9677939]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1092937, 1008714, 12132312, 1075979, 998206, ...","[1092937, 1008714, 12132312, 1075979, 998206]","[12524016, 10456568, 1092502, 1092937, 1008714]"


#### Вычисляем метрики на data_val_lvl_2

In [524]:
print(' precision@k для модели первого уровня (data_val_lvl_2):', get_precision_at_k(result_lvl_2, 'model1', K), '\n',
      'precision@k для модели второго уровня (data_val_lvl_2):', get_precision_at_k(result_lvl_2, 'model2', K))

 precision@k для модели первого уровня (data_val_lvl_2): 0.15563173359451518 
 precision@k для модели второго уровня (data_val_lvl_2): 0.4721759712699968


### Получим расчет для data_test

А теперь делаем почти то же самое, только обучим первую модель на полном датасете data_train и cделаем предсказание кандидатов для data_test. Затем полученных кандидатов ранжируем второй моделью и получим предсказания для data_test.

In [525]:
data_test = pd.read_csv('../raw_data/retail_test.csv')

In [526]:
# Первичная фильтрация данных
data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)
# Обучение модели первого уровня
recommender = MainRecommender(data_train)
# Получаем список актуальных покупок и кандидатов для data_test
result_lvl = get_actual(data_test)
result_lvl = get_candidates(recommender, result_lvl)
# Формируем данные для второй модели
X = get_X_data(result_lvl)
# Фичи
X = featuring(data_test, X)
# Предобработка данных перед предсказанием модели 2
X = prepare_X(X)
# Берем топ-k предсказаний второй модели для data_test, ранжированных по вероятности, для каждого юзера
model2 = get_model2(lgb, X, K)
# Находим рекомендации по первой модели
model1 = get_model1(recommender, result_lvl, K)
# Сливаем всё в одну таблицу
result_lvl = result_lvl.merge(model1, on='user_id', how='left')
result_lvl = result_lvl.merge(model2, on='user_id', how='left')
result_lvl.head(2)

,user_id,actual,candidates,model1,model2
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[856942, 9297615, 5577022, 8293439, 9655212, 8...","[856942, 9297615, 5577022, 8293439, 9655212]","[9527558, 991024, 5582712, 1087268, 12352054]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[1103898, 911974, 1076580, 5567582, 1007414, 1...","[1103898, 911974, 1076580, 5567582, 1007414]","[1139142, 885023, 12263492, 5567582, 911974]"


#### Вычисляем метрики на data_test

In [527]:
print(' precision@k для модели первого уровня (data_test):', get_precision_at_k(result_lvl, 'model1', K), '\n',
      'precision@k для модели второго уровня (data_test):', get_precision_at_k(result_lvl, 'model2', K))

 precision@k для модели первого уровня (data_test): 0.14068965517241377 
 precision@k для модели второго уровня (data_test): 0.4432979664014147


#### Результирующая метрика precision@5 с использованием 2-х моделей для тестовых данных равна 0.4432979664014147

#### Сохранение результатов

In [528]:
result_lvl[['user_id', 'model2']].to_csv('recommendations.csv', index=False)